# 4.8 Task

### This script contains the following points:

#### Aggregated mean
#### Creating flag variables

In [2]:
#import libraries
import pandas as pd
import numpy as np
import os

In [3]:
#import data
path = '/Users/kimballwightman/Documents/CareerFoundry/Instacart Basket Analysis'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_derive.pkl'))

In [8]:
ords_prods_merge.head()

,order_id,user_id,user_order_num,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-Range Product,Regularly Busy,Regularly Busy,Average Orders
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Regularly Busy,Least Busy Days,Average Orders
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Regularly Busy,Least Busy Days,Most Orders
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least Busy,Least Busy Days,Average Orders
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least Busy,Least Busy Days,Most Orders


In [15]:
#aggregated mean of user_order_num groubed by department_id
ords_prods_merge.groupby('department_id').agg({'user_order_num':['mean']})

,user_order_num
,mean
department_id,
1,15.461415
2,17.280527
3,17.170316
4,17.814687
5,15.252884
6,16.449556
7,17.215801
8,15.354308


In [16]:
#find aggregated mean of user_order_num for the whole dataframe
ords_prods_merge['user_order_num'].mean()

17.145811399458328

The average number of orders for a customer is around 17 orders. When we look at this average when split by department_id, we can see there are 6 departments that meet the lower end of the spectrum for the average, around 15-16 orders. There are two which are far above the average: bulk (around 20 orders) and missing (around 23 orders per customer)

In [17]:
#create a loyalty flag
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['user_order_num'].transform(np.max)

In [18]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [21]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular Customer    15719388
Loyal Customer      10187688
New Customer         6182747
Name: loyalty_flag, dtype: int64

In [24]:
#check basic stats of product prices to see customer type spending habits
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal Customer    10.411196  1.0  99999.0
New Customer      13.317945  1.0  99999.0
Regular Customer  12.496123  1.0  99999.0

As we can see above, New customers spend the most on average for their orders. As a customer continues to make more orders, the average price they pay drops, shown by Regular customers spending less than New customers, and Loyal customers spending the least on average of all customers.

In [25]:
#create a spending flag
ords_prods_merge['avg_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [27]:
ords_prods_merge.loc[ords_prods_merge['avg_spend'] < 10, 'spending_flag'] = 'Low Spender'

In [28]:
ords_prods_merge.loc[ords_prods_merge['avg_spend'] >= 10, 'spending_flag'] = 'High Spender'

In [31]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low Spender     31460479
High Spender      629344
Name: spending_flag, dtype: int64

In [32]:
#create a frequency flag
ords_prods_merge['median_dspo'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [33]:
ords_prods_merge.loc[ords_prods_merge['median_dspo'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'

In [34]:
ords_prods_merge.loc[(ords_prods_merge['median_dspo'] <= 20) & (ords_prods_merge['median_dspo'] > 10), 'frequency_flag'] = 'Regular Customer'

In [35]:
ords_prods_merge.loc[ords_prods_merge['median_dspo'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [36]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent Customer        21347904
Regular Customer          7142755
Non-Frequent Customer     3599159
NaN                             5
Name: frequency_flag, dtype: int64

In [38]:
#export data
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data','ords_prods_group.pkl'))